# **⚡ Simulated Annealing x Mixing Properties**

In [1]:
import numpy as np

## **⭕ Lennard Jones Potential**

In [2]:
# Lennard-Jones potential function
def lj_potential(positions, epsilon=1.0, sigma=1.0):
    n_particles = positions.shape[0]
    potential = 0.0
    for i in range(n_particles):
        for j in range(i + 1, n_particles):
            r = np.linalg.norm(positions[i] - positions[j])
            if r != 0:
                sr6 = (sigma / r) ** 6
                sr12 = sr6 ** 2
                potential += 4 * epsilon * (sr12 - sr6)
    return potential

## **🤖 Algorithm 11.1**

In [5]:
def optimize_lj_tmcmc_enhanced(n_particles=5, N=10000, epsilon=1.0, sigma=1.0):
    theta = np.random.uniform(-1.0, 1.0, (n_particles, 3))
    best_theta = theta.copy()
    best_energy = lj_potential(theta, epsilon, sigma)

    a1 = 0.1
    history = []

    for t in range(1, N + 1):
        # ------- Stage 1: Basic TMCMC Move -------
        e1 = np.random.uniform(low=-1.0, high=1.0, size=(n_particles, 3))
        e1[np.abs(e1) < 0.01] = 0.01 * np.sign(e1[np.abs(e1) < 0.01])

        b = np.random.choice([-1, 1], size=(n_particles, 3))
        U1 = np.random.uniform()

        if U1 < 0.5:
            theta_tilde = theta + b * a1 * np.abs(e1)
            J = 1.0  # Additive Jacobian
        else:
            theta_tilde = theta * np.exp(b * a1 * np.abs(e1))
            J = max(np.abs(e1).sum(), 1e-8)  # Avoid log(0)

        E_current = lj_potential(theta, epsilon, sigma)
        E_tilde = lj_potential(theta_tilde, epsilon, sigma)

        ell_diff1 = -E_tilde + E_current
        tau_t = 1 / np.log(t + 1) if t >= 100 else 1 / (t + 1)
        expo1 = (ell_diff1 + np.log(J)) / tau_t
        expo1 = np.clip(expo1, -700, 700)  # Avoid overflow
        alpha1 = min(1.0, np.exp(expo1))

        if np.random.rand() < alpha1:
            theta = theta_tilde
            if E_tilde < best_energy:
                best_energy = E_tilde
                best_theta = theta.copy()

        # ------- Stage 2: Further Mixing -------
        e3 = np.random.uniform(low=-1.0, high=1.0, size=(n_particles, 3))
        e3[np.abs(e3) < 0.01] = 0.01 * np.sign(e3[np.abs(e3) < 0.01])
        U2 = np.random.uniform()

        if U2 < 0.5:
            u1 = np.random.uniform()
            if u1 < 0.5:
                theta_star = theta + a1 * np.abs(e3)
            else:
                theta_star = theta - a1 * np.abs(e3)
            J = 1.0
        else:
            e4 = np.random.choice([-1, 1], size=(n_particles, 3)) * 0.5
            e4[np.abs(e4) < 1e-8] = 1e-8  # avoid division by zero

            u2 = np.random.uniform()
            if u2 < 0.5:
                theta_star = theta * e4
                J = max(np.abs(e4).sum(), 1e-8)
            else:
                theta_star = theta / e4
                J = max((1 / np.abs(e4)).sum(), 1e-8)

        E_star = lj_potential(theta_star, epsilon, sigma)
        ell_diff2 = -E_star + lj_potential(theta, epsilon, sigma)
        tau_t2 = 1 / np.log(t + 1) if t >= 100 else 1 / (t + 1)
        expo2 = (ell_diff2 + np.log(J)) / tau_t2
        expo2 = np.clip(expo2, -700, 700)
        alpha2 = min(1.0, np.exp(expo2))

        if np.random.rand() < alpha2:
            theta = theta_star
            if E_star < best_energy:
                best_energy = E_star
                best_theta = theta.copy()

        history.append(best_energy)

    return best_theta, best_energy, history

## **🧪 Testing**

In [6]:
# Run the enhanced optimizer
best_positions, min_energy, energy_history = optimize_lj_tmcmc_enhanced(n_particles=10, N=5000)

# Print results
print("Best particle positions:\n", best_positions)
print("Minimum Lennard-Jones potential:", min_energy)

Best particle positions:
 [[ 1.34954068  0.12483248 -1.57149937]
 [-1.5530553  -1.31539608 -1.06625568]
 [-1.35814854  0.1198172  -0.51412063]
 [-1.39545953 -1.97628077  2.21418743]
 [ 1.29268017  1.84356122 -0.96964936]
 [-0.57726945 -1.30703035  0.83783578]
 [-0.43498149  1.28577819  1.87820177]
 [ 2.16890387 -1.74348265 -0.78803318]
 [-1.24143804  0.57307596  1.60692705]
 [ 0.53920094 -0.16356946  1.18031284]]
Minimum Lennard-Jones potential: -2.1799939678437457
